In [ ]:
#update to be relative to your project structure
main_input_folder_path = r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows"

import sys
import os
import pandas as pd

# Add the src directory to the system path and import functions
sys.path.append(os.path.abspath('../src'))
from schedule_weather import generate_activity_list, schedule_activities, scheduled_df, shift_start_end
from plot import plot

In [ ]:
#read in activity data from excel
path = os.path.join(main_input_folder_path, "WeatherWindows.xlsx")
act_df = pd.read_excel(path, sheet_name="Sheet1", skiprows=1)
act_df.info()
activities = generate_activity_list(act_df)


In [ ]:


# Load weather/tide data
weather_df = pd.read_csv(
    r"C:\Users\webbb\VolkerWessels UK\Grp_VMS GeoSeacore Collaboration Space - SZC\2 Internal WIP\06. MST's\Weather Windows\scripts\combined_tide_data_with_rate.csv"
)

# Parse Time column
weather_df['Time'] = pd.to_datetime(weather_df['Time'])

# Ensure absolute rate for current proxy
weather_df['Tide_rate_m_per_hr_abs'] = weather_df['Tide_rate_m_per_hr'].abs()

# Define start date for hour numbering
start_date = weather_df['Time'].min()
weather_df['Hour_index'] = (weather_df['Time'] - start_date).dt.total_seconds() / 3600

# Build weather_data dictionary for scheduling
weather_data = {
    int(round(row['Hour_index'] / 0.5)): {  # 0.5 for 30-min interval
        "tidal_level": row['Tide Prediction (m)'],
        "tidal_current": row['Tide_rate_m_per_hr_abs']
    }
    for _, row in weather_df.iterrows()
}

print(weather_data)


# Schedule with 30-minute intervals
scheduled_data = schedule_activities(activities, weather_data, analysis_interval=1)


df = scheduled_df(scheduled_data)
df = shift_start_end(df)

print(df.head())